# SIRI Realtime without position using GTFS 
Ruter and others do not return position in lat/long only what stations it has passed and has to pass and time to station.
Could we get approx position using GTFS data shapes/stops?

In [ ]:
import pandas as pd
import requests
from lxml import etree

In [ ]:
nsmap = {
    "siri": "http://www.siri.org.uk/siri",
    "ns2": "http://www.ifopt.org.uk/acsb",
    "ns3": "http://www.ifopt.org.uk/ifopt",
    "ns4": "http://datex2.eu/schema/2_0RC1/2_0",
}

In [ ]:
resp = requests.get("https://api.entur.io/realtime/v1/rest/et?datasetId=RUT")
assert resp.ok

In [ ]:
resp.content[0:1000]

In [ ]:
prstree = etree.fromstring(resp.content)
root = prstree.tag

all_items = []

for journey in prstree.iter(etree.QName(nsmap["siri"], "EstimatedVehicleJourney")):
    journey_level1_dict = {
        etree.QName(x).localname: x.text for x in journey if x.text is not None
    }
    framed_vehicle_ref = journey.find(
        etree.QName(nsmap["siri"], "FramedVehicleJourneyRef")
    )
    framed_vehicle_dict = {
        etree.QName(x).localname: x.text
        for x in framed_vehicle_ref
        if x.text is not None
    }
    journey_dict = journey_level1_dict | framed_vehicle_dict
    for estimated in journey.iter(etree.QName(nsmap["siri"], "EstimatedCall")):
        estimated_dict = {
            etree.QName(x).localname: x.text for x in estimated
        } | journey_dict
        estimated_dict["XType"] = "EstimatedCall"
        all_items.append(estimated_dict)
    for recorded in journey.iter(etree.QName(nsmap["siri"], "RecordedCall")):
        recorded_dict = {
            etree.QName(x).localname: x.text for x in recorded
        } | journey_dict
        recorded_dict["XType"] = "RecordedCall"
        all_items.append(recorded_dict)

df_raw = pd.DataFrame(all_items).convert_dtypes(dtype_backend="pyarrow")
df_raw.sample(10).T

In [ ]:
df = df_raw.assign(
    Order=df_raw.Order.astype("int64[pyarrow]"),
    AimedDepartureTime=pd.to_datetime(df_raw.AimedDepartureTime, format="ISO8601"),
    ActualDepartureTime=pd.to_datetime(df_raw.ActualDepartureTime, format="ISO8601"),
    ExpectedDepartureTime=pd.to_datetime(
        df_raw.ExpectedDepartureTime, format="ISO8601"
    ),
    ExpectedArrivalTime=pd.to_datetime(df_raw.ExpectedArrivalTime, format="ISO8601"),
    RecordedAtTime=pd.to_datetime(df_raw.RecordedAtTime, format="ISO8601"),
)

In [ ]:
(
    df.query("XType == 'EstimatedCall'")
    .sort_values("Order")
    .groupby(["DatedVehicleJourneyRef"])
    .first()
    .T
)

In [ ]:
(
    df.query("Order > 1")
    .query("XType == 'RecordedCall'")
    .sort_values("Order")
    .groupby(["DatedVehicleJourneyRef"])
    .last()
    .T
)